Ada López del Castillo Avilés, NIU:1605347

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
)
from sklearn.metrics import matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Mesures de puresa (per coherència amb els apunts)
def entropy(p):
    p = np.asarray(p, dtype=float)
    p = p[p > 0]
    return -np.sum(p * np.log2(p))

def gini(p):
    p = np.asarray(p, dtype=float)
    return 1.0 - np.sum(p**2)

def error_rate(p):
    p = np.asarray(p, dtype=float)
    return 1.0 - np.max(p)

# Mètriques d'equitat (grupal) per atribut sensible A (0=desfavorit, 1=afavorit)
# y_true i y_pred són binàries (0/1)

def statistical_parity_difference(y_pred, A):
    # SPD = |P(ŷ=1|A=1) - P(ŷ=1|A=0)|
    p1 = (y_pred[A==1] == 1).mean() if np.any(A==1) else 0.0
    p0 = (y_pred[A==0] == 1).mean() if np.any(A==0) else 0.0
    return abs(p1 - p0)


def statistical_parity_ratio(y_pred, A, eps=1e-12):
    # SPR = P(ŷ=1|A=1) / P(ŷ=1|A=0)  (definim 0 si denominador=0 i numerador>0)
    p1 = (y_pred[A==1] == 1).mean() if np.any(A==1) else 0.0
    p0 = (y_pred[A==0] == 1).mean() if np.any(A==0) else 0.0
    if p0 < eps and p1 < eps:
        return 1.0  # ambdues 0 → paritat trivial
    if p0 < eps:
        return 0.0
    return p1 / p0


def equal_opportunity_difference(y_true, y_pred, A):
    # EOD = |TPR(A=1) - TPR(A=0)|, on TPR = P(ŷ=1|Y=1, A)
    mask1 = (y_true==1) & (A==1)
    mask0 = (y_true==1) & (A==0)
    tpr1 = (y_pred[mask1] == 1).mean() if np.any(mask1) else 0.0
    tpr0 = (y_pred[mask0] == 1).mean() if np.any(mask0) else 0.0
    return abs(tpr1 - tpr0)


def predictive_equality_difference(y_true, y_pred, A):
    # PED = |FPR(A=1) - FPR(A=0)|, FPR = P(ŷ=1|Y=0, A)
    mask1 = (y_true==0) & (A==1)
    mask0 = (y_true==0) & (A==0)
    fpr1 = (y_pred[mask1] == 1).mean() if np.any(mask1) else 0.0
    fpr0 = (y_pred[mask0] == 1).mean() if np.any(mask0) else 0.0
    return abs(fpr1 - fpr0)


def group_confusions(y_true, y_pred, A):
    # Taules de confusió per grup
    return {
        g: confusion_matrix(y_true[A==g], y_pred[A==g], labels=[0,1]) if np.any(A==g) else np.zeros((2,2), dtype=int)
        for g in [0,1]
    }

## Exercici 1: Càlcul manual de mètriques d'equitat



Considera la taula següent (construïda sobre el conjunt test d'un model):
- Grup A=0 (desfavorit): TP=40, FP=20, TN=110, FN=30
- Grup A=1 (afavorit): TP=55, FP=15, TN=115, FN=15

Calcula TPR i FPR per a cada grup. 
Calcula EOD i PED. 
Si el model donava (P(\hat{Y}=1|A=0)=\frac{TP+FP}{N_0}) i (P(\hat{Y}=1|A=1)=\frac{TP+FP}{N_1}), calcula SPD i SPR.

Pista: (N_g = TP+FP+TN+FN) per cada grup. Definicions segons Tema 7 de teoria.

In [ ]:
#Posem les dades del grup 0 i 1, i després les classifiquem en predictives i observades

TP0, FP0, TN0, FN0 = 40, 20, 110, 30 
TP1, FP1, TN1, FN1 = 55, 15, 115, 15  

A_0 = np.zeros(TP0 + FP0 + TN0 + FN0, dtype=int)
y_true_0 = np.array([1]*(TP0+FN0) + [0]*(FP0+TN0), dtype=int)
y_pred_0 = np.array([1]*TP0 + [0]*FN0 + [1]*FP0 + [0]*TN0, dtype=int)

A_1 = np.ones(TP1 + FP1 + TN1 + FN1, dtype=int)
y_true_1 = np.array([1]*(TP1+FN1) + [0]*(FP1+TN1), dtype=int)
y_pred_1 = np.array([1]*TP1 + [0]*FN1 + [1]*FP1 + [0]*TN1, dtype=int)

#Ajuntem els dos grups, la informació (seria com construir la matriu confusió manualment)
A = np.concatenate([A_0, A_1])
y_true = np.concatenate([y_true_0, y_true_1])
y_pred = np.concatenate([y_pred_0, y_pred_1])

#Calcul de TPR i FPR de cada grup
def tpr_fpr(y_true, y_pred, A, g):
    # TPR = P(ŷ=1 | Y=1, A=g)
    mask_pos = (y_true == 1) & (A == g)
    tpr = (y_pred[mask_pos] == 1).mean() if np.any(mask_pos) else 0.0

    # FPR = P(ŷ=1 | Y=0, A=g)
    mask_neg = (y_true == 0) & (A == g)
    fpr = (y_pred[mask_neg] == 1).mean() if np.any(mask_neg) else 0.0
    return tpr, fpr

#Guardem els resultats i els imprimim
TPR0, FPR0 = tpr_fpr(y_true, y_pred, A, g=0)
TPR1, FPR1 = tpr_fpr(y_true, y_pred, A, g=1)

print("TPR0 =", TPR0, "i FPR0 =", FPR0)
print("TPR1 =", TPR1, "i FPR1 =", FPR1)

#Ara calculem EOD i PED i els imprimim
EOD = equal_opportunity_difference(y_true, y_pred, A)
PED = predictive_equality_difference(y_true, y_pred, A)

print("\nEOD =", EOD)
print("PED =", PED)

# Per últim calculem SPD i SPR
SPD = statistical_parity_difference(y_pred, A)
SPR = statistical_parity_ratio(y_pred, A)

print("\nSPD =", SPD)
print("SPR =", SPR)


Primer s'han construit les matrius de confusió de cada grup i s'ha calculat TPR i FPR de cadascun.
Amb aquestes dades s'han calculat les mètriques d'equitat, EOD=0.214 que és una diferència important i indica que el model està predint més falsos negatius en el cas del grup A=0, per tant no detecta bé els positius reals.

En el cas del PED=0.038 surt un valor molt més petit, el qual indica que el model està patint falsos positius de manera similar en els dos grups, tot i que el A=1 segueix tenint una taxa lleugerament millor.

A més s'ha calculat el SPD i el SPR que indiquen que el model assigna més prediccions positives al grup A=1, per tant podem concloure que el model afavoreix al grup A=1.

Per tant, el problema està en els falsos negatius del grup A=0, que és on realment hi ha una diferència important comparat amb el grup A=1.

### Exercici 2: Tuning de k al k-NN i compromís equitat–rendiment

Usant el dataset sintètic:

- Avalua k-NN per k ∈ {1,3,5,7,11,15,21} amb 5-fold CV (estratificada).
- Dibuixa ACC i SPD/EOD en funció de k.
- Escull k sota el criteri: minimitzar SPD subjecte a ACC ≥ (ACC màxim − 0.02).

Pista: Recorda estandarditzar features per a k-NN.

In [ ]:
from scipy.special import expit

n = 5000
A = np.random.binomial(1, 0.5, size=n)
X1 = np.random.normal(0.5*A, 1.0, size=n)
X2 = np.random.normal(0.3*A, 1.0, size=n)  # segon predictor correlacionat
X = np.c_[X1, X2]

# Probabilitat positiva amb model logístic (beta0=0, beta=[1,1])
logit = 0.0 + 1.0*X1 + 1.0*X2
p = expit(logit)
Y = np.random.binomial(1, p)

synth = pd.DataFrame({"X1":X1, "X2":X2, "A":A, "Y":Y})
synth.head()

In [ ]:
#Fem una funció que avalui Knn pels diferents valors donats i que calculi ACC; EOD; SPD
k_valors= [1, 3, 5, 7, 11, 15, 21]
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

rows = []

for k in k_valors:
    acc_folds = []
    spd_folds = []
    eod_folds = []

    model = Pipeline([
        ("scaler", StandardScaler()),
        ("knn", KNeighborsClassifier(n_neighbors=k))
    ])

    for train_idx, test_idx in cv.split(X, Y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = Y[train_idx], Y[test_idx]
        A_test = A[test_idx]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        spd = statistical_parity_difference(y_pred, A_test)
        eod = equal_opportunity_difference(y_test, y_pred, A_test)

        acc_folds.append(acc)
        spd_folds.append(spd)
        eod_folds.append(eod)

    rows.append({
        "k": k,
        "ACC": np.mean(acc_folds),
        "SPD": np.mean(spd_folds),
        "EOD": np.mean(eod_folds),
    })

resultats = pd.DataFrame(rows).sort_values("k")
print(resultats)

#Representem els valors en funció de k en un gràfic
plt.plot(resultats["k"], resultats["ACC"], marker="o")
plt.xlabel("k")
plt.ylabel("Accuracy (mitjana CV)")
plt.title("ACC vs k (k-NN, 5-fold CV)")
plt.grid(True)
plt.show()

plt.figure()
plt.plot(resultats["k"], resultats["SPD"], marker="o", label="SPD")
plt.plot(resultats["k"], resultats["EOD"], marker="o", label="EOD")
plt.xlabel("k")
plt.ylabel("Mètrica d'equitat (mitjana CV)")
plt.title("Equitat vs k (k-NN, 5-fold CV)")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# ACC màxima
acc_max = resultats["ACC"].max()

# Restricció: ACC ≥ ACC_max − 0.02
candidats = resultats[resultats["ACC"] >= acc_max - 0.02]

# Minimitzem SPD sota la restricció donada
k_opt = candidats.sort_values("SPD").iloc[0]["k"]

#Imprimim el valor de la ACC màxima, els candidats i finalment k escollit
print("ACC màxima:", acc_max)
print("Candidats:")
print(candidats)
print("k seleccionat:", k_opt)

S'ha avaluat el model KNN amb els diferents valors donats amb CV 5 i s'han estandaritzat les dades tal i com es demanava, després s'ha mesurat ACC, SPD i EOD amb cadascuna d'elles i s'ha imprés per pantalla, després s'ha dibuixat per a una millor comprensió.
A la gràfica, s'observa que l'accuracy va augmentant a mesura que k creix, això és degut a que en el model Knn normalment amb k petit el model és més sorollós i poc estable i a mesura que anem augmentant la k, es va estabilitzant i millorant el rendiment.
Ara, la gràfica de EOD i SPD veiem que el SPD va empitjoran a mesura que la k augmenta (SPD augmenta també), això indica que en pujar la k, el percentatge de prediccions positives es separa més entre grups. I pel que fa al EOD, també creix a mesura que augmenta la k.

Amb tota aquesta informació, s'ha usat un criteri en concret: minimitzar SPD subecte a ACC>= ACC max -0.02 per tal d'escollir la millor k per aquest dataset, i ens ha sortit k=7, és a dir, un valor entre mig de tots, això té sentit perquè hem vist que al pujar la k, millorem el rendiment pero perdem equitat (augmenta SPD), es per això que el valor k=7 ha sortit el millor.

### Exercici 3: Profunditat de l'arbre i equitat

Avaluarem DecisionTreeClassifier amb max_depth ∈ {1..10} sobre el dataset sintètic.

- Representa ACC i SPD en funció de la profunditat.
- Identifica el punt on l'increment de profunditat no millora ACC però empitjora l'equitat (increment de SPD).

In [ ]:
#Fem el mateix que en l'anterior exercici però calculant ACC I SPD en DecisionTree
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

depths = range(1, 11)
rows = []

for d in depths:
    accs, spds = [], []
    clf = DecisionTreeClassifier(max_depth=d, criterion="gini", random_state=42)

    for tr, te in cv.split(X, Y):
        clf.fit(X[tr], Y[tr])
        yhat = clf.predict(X[te])

        accs.append(accuracy_score(Y[te], yhat))
        spds.append(statistical_parity_difference(yhat, A[te]))  # <-- IMPORTANT

    rows.append([d, np.mean(accs), np.mean(spds)])

resultats = pd.DataFrame(rows, columns=["depth", "ACC", "SPD"])

#
plt.figure()
plt.plot(resultats.depth, resultats.ACC, marker='o')
plt.xlabel("max_depth")
plt.ylabel("Accuracy (mitjana CV)")
plt.show()

plt.figure()
plt.plot(resultats.depth, resultats.SPD, marker='o')
plt.xlabel("max_depth")
plt.ylabel("SPD (mitjana CV)")
plt.show()


In [ ]:
# Diferències entre profunditats consecutives
resultats["ACC_diff"] = resultats["ACC"].diff()
resultats["SPD_diff"] = resultats["SPD"].diff()

# Punt on augmentar profunditat ja no millora ACC però empitjora equitat (SPD)
tol = 0.001
problematic = resultats[(resultats["ACC_diff"] <= tol) & (resultats["SPD_diff"] > 0)]

print(resultats.round(4))
print("\nPunts problemàtics (ACC no millora > tol i SPD empitjora):")
print(problematic.round(4))


S'ha avaluat un DecisionTree canviant la profunditat a diversos valors sobre el mateix dataset que l'anterior exercici, i s'ha mesurat l'accuracy i el SPD en cadascuna de les profunditats.
El que s'ha observat és que el rendiment millorava al usar profunditats més elevades (del 1 al 4 hi ha una millora de rendiment), però passada una profunditat (la 4), el rendiment torna a decaure de manera molt lleugera, això pot ser degut a un overfitting i pèrdua de generalització.
Pel que fa a la SPD els valors van canviant sense un patró clar, però si que és veu q en aporfunidir més, l'equitat empitjora.

Després s'ha impres per pantalla els resultats numèrics de totes les profunditats i s'ha escollit el punt on en augmentar la profunditat no millora l'ACC i augmenta SPD, i ha del 4 al 5, fent així que sigui el punt on convé aturar-se.
Per tant fer un arbre més complex no garanteix que sigui millor.

### Exercici 4: Probabilitats amb SVM (Platt) i efecte en l'equitat

1. Ajusta una SVM lineal al dataset sintètic i obtén probabilitats calibrades (probability=True aplica una aproximació tipus Platt scaling).
2. Compara mètriques d'equitat i AUC amb i sense calibració (aprox. usar SVC(..., probability=True) vs decision_function (sigmoide manual)).
3. Discutiu: millorar la calibració del score ajuda a fixar llindars que milloren el compromís equitat–rendiment?

In [ ]:
#Fem una SVM lineal
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

vals = []

for tr, te in cv.split(X, Y):
    Xtr, Xte = X[tr], X[te]
    ytr, yte = Y[tr], Y[te]
    Ate = A[te]

    #Estandaritzem les dades com sempre
    sc = StandardScaler()
    Xtr_s = sc.fit_transform(Xtr)
    Xte_s = sc.transform(Xte)

    #SVM calibrada
    svm_calibrada = SVC(kernel="linear", probability=True, random_state=42)
    svm_calibrada.fit(Xtr_s, ytr)
    p_cal = svm_calibrada.predict_proba(Xte_s)[:, 1]
    yhat_cal = (p_cal >= 0.5).astype(int)

    #SVM no calibrada
    svm_no_cal = SVC(kernel="linear", probability=False, random_state=42)
    svm_no_cal.fit(Xtr_s, ytr)
    score = svm_no_cal.decision_function(Xte_s)
    #sigmoide
    p_no_cal = expit(score)          
    yhat_no_cal = (p_no_cal >= 0.5).astype(int)

    vals.append([
        accuracy_score(yte, yhat_cal),
        roc_auc_score(yte, p_cal),
        statistical_parity_difference(yhat_cal, Ate),
        equal_opportunity_difference(yte, yhat_cal, Ate),
        predictive_equality_difference(yte, yhat_cal, Ate),

        accuracy_score(yte, yhat_no_cal),
        roc_auc_score(yte, score),  
        statistical_parity_difference(yhat_no_cal, Ate),
        equal_opportunity_difference(yte, yhat_no_cal, Ate),
        predictive_equality_difference(yte, yhat_no_cal, Ate),
    ])

res = pd.DataFrame(vals, columns=[
    "ACC_cal","AUC_cal","SPD_cal","EOD_cal","PED_cal",
    "ACC_no_cal","AUC_no_cal","SPD_no_cal","EOD_no_cal","PED_no_cal"
])

summary = pd.DataFrame({
    "metrica": ["ACC","AUC","SPD","EOD","PED"],
    "SVM_calibrada": [
        res["ACC_cal"].mean(), res["AUC_cal"].mean(),
        res["SPD_cal"].mean(), res["EOD_cal"].mean(), res["PED_cal"].mean()
    ],
    "SVM_no_cal": [
        res["ACC_no_cal"].mean(), res["AUC_no_cal"].mean(),
        res["SPD_no_cal"].mean(), res["EOD_no_cal"].mean(), res["PED_no_cal"].mean()
    ]
})

print(summary.round(4))

S'ha entrenat una SVM lineal sobre el dataset sintètic i s'han comparat dues maneres d'obtenir probabilitats, la calibrada i la no calibrada. I s'han comparat ACC, AUC, les mètriques de equitat SPD, EOD, PED amb un llindar de 0.5.
Notem que l'accuracy canvia molt poc, hi ha el mateix rendiment, i AUC és exactament igual.
Ara, en quant a les de equitat si que hi ha una mica més de canvi, la no calibrada dona millors valors en les 3 mètriques (però la diferència és molt petita).

Per tant, tot i que la calibració no ajuda a millorar l'equitat amb aquest llindar, calibrar és útil ja que els resultats són més interpretables. En aquest cas no calia calibrar, però la calibració és interessant si volem ajustar el llindar de decisió de manera controlada per optimitzar el compromís equitat–rendiment.